> # Traitement Automatique de texte en IA projet
> 
> > Antoine Vidal-Mazuy - Yann Brault
> > 28/12/2021 Université Côtes d'azur

<br>

> ## Introduction

<br>

> blablzqdbd bzqjkdb qz

<br>
<br>

> ## Datasets utilisés
>
> > #### Datasets de base:

<br>

> Nous avons décidé de construire nous même notre base de donnée. Pour cela nous avons conçu un petit scraper su site Jeux-Video.com. <br>
> Le code source du scraper se trouve dans le lien suivant [JVCScraper](https://github.com/Brotherta/JVCScraper).

<br>

> Nous avons récupéré tous les commentaires et notes d'utilisateurs sur environ 50 jeux, pour un total de 102000 avis. Puis nous les avons sauvegarder sous forme de fichiers csv. <br>
> Nous avons dû nettoyer les avis de tous les charactères spéciaux, des mots de liaisons, les urls et tout ce qui n'apportait rien à la compréhension de l'avis.
> Afin de nettoyer plus facilement les commentaires nous avons créer une classe **CleanData** qui nous permettait d'effectuer plusieurs tâches de pré-processing.

In [12]:
import re
from spellchecker import SpellChecker

import pandas as pd
from pandas.core.frame import DataFrame
from tqdm import trange



class CleanData:

    def __init__(self, max_words, df: DataFrame = None) -> None:
        self.max_words = max_words
        self.df = df
        self.unused_chars = ',|;|\&|\#|\@|\%|\:|\>|\<|\(|\)|\{|\}|\=|\+|\_|\[|\}|\^|\*|\!|\?|\/|\¨|\~|\\\|\§|\||[0-9]|\[|\]|\"'
        self.connecting_words = [
            "c'est", "ces", "ses", "s'est", "a", "de", "du", 
            "et", "le", "les", "un", "une", "pour", "sur", "etc", "est", "c",
            'la', "jeu", "que", "des", "en", "ce", "qu", "ca", "y", "je", "sa", "son",
            "au", "ai", "mon", "ma", "mes", "qui", "je", "tu", "il", "ils", "elles", "elle", "vous", "nous",
            "qu'il", "qu'elle", "qu'ils", "qu'elles", "qu'on",
            "on", "se", "par"]
        self.urls = r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%|\-)*\b'
            
        self.spell = SpellChecker(language='fr')

    def correction_spelling(self, review):
        """ 
        try to elimiminates the unknown words of a sentence, and 
        replacing it by a correct word. 
        """
        
        review_list = review.split(" ")
        
        bad = []
        for word in review_list:
            if word != " ":
                bad = self.spell.unknown(review_list)

        new_list = []
        for word in review_list:
            if word in bad:
                new_list.append(self.spell.correction(word))
            else:
                new_list.append(word)
        
        return ' '.join(new_list)

    def replace_nan(self):
        """
        Replace nan by 'bon' or 'mauvais' in the dataframe.
        """
        to_drop = []
        for i in self.df.index:
            r = self.df['avis'][i]
            if pd.isna(r) or r in ['nan', 'Nan'] or type(r) == float:
                to_drop.append(i)
        print("droped nan : ", len(to_drop))
        self.df = self.df.drop(to_drop)

    def remove_urls(self, review):
        review = re.sub(self.urls, '', review, flags=re.MULTILINE)
        return(review)

    def clean_str(self, review):
        """
        Remove special characters from the string.
        """

        if len(review) > 0 or review != None:
            review = re.sub(self.unused_chars, ' ', review)
            review = review.replace('.', ' ').replace('\t', ' ').replace('\r', ' ').replace('\n', ' ')
            review = review.lower()
            review = re.sub(' +', ' ', review)
            review = re.sub(r' (?! ) ', '', review) #removing single characters
       
        return review
    
    def clean_stop_words(self, review):
        review_list = review.split(" ")

        new_list = []
        for word in review_list:
            if word != " " and not word in self.connecting_words and len(word) > 1:
                new_list.append(word)

        return ' '.join(new_list)

    def clean_review(self, review):
        review = self.clean_str(review)
        review = self.clean_stop_words(review)
        review = self.correction_spelling(review)
        
        return review

    def clean_dataset(self):
        """
        Main method call to prepare a text to be vectorized.
        """

        self.df = self.replace_nan()
        for i in trange(self.df.shape[0]):
            review = self.df.at[i, 'avis']
            review = self.clean_str(review)
            review = self.clean_stop_words(review)
            review = self.correction_spelling(review)
                
            self.df.at[i, 'avis'] = review

    def filter_long_review(self):
        """
        Filter the string with too many words.
        """

        to_drop = []
        for i in self.df.index:
            review = self.df['avis'][i]
            review_list = review.split()

            if len(review_list) > self.max_words:
                to_drop.append(i)

        self.df = self.df.drop(to_drop)
        print(f"dropped {len(to_drop)} lines")

    def fix_repartition_for_4_classes(self):
        """
        Fix the bad repartitions of the dataset, by removing randomly good reviews.
        """

        d = self.df.groupby(['classe_bon_mauvais'], as_index=False).count()
        nb_bad = d['avis'][0]

        nb_good = d['avis'][2]
        to_remove = nb_good - nb_bad

        while(to_remove > 0):
            row: DataFrame = self.df.sample()
            index = row.first_valid_index()
            print(f"{to_remove}")

            if row['classe_bon_mauvais'][index] == 2:
                self.df.drop(index, inplace=True)
                to_remove -= 1


        nb_good = d['avis'][3]
        to_remove = nb_good - nb_bad

        while(to_remove > 0):
            row: DataFrame = self.df.sample()
            index = row.first_valid_index()
            print(f"{to_remove}")

            if row['classe_bon_mauvais'][index] == 3:
                self.df.drop(index, inplace=True)
                to_remove -= 1
        
        d = self.df.groupby(['classe_bon_mauvais'], as_index=False).count()
        nb_bad = d['avis'][0]
        nb_good = d['avis'][1]
        print("2", nb_bad, nb_good)

    def fix_repartition(self):
        """
        Fix the bad repartitions of the dataset, by removing randomly good advice.
        """

        d = self.df.groupby(['classe_bon_mauvais'], as_index=False).count()
        nb_bad = d['avis'][0]
        nb_good = d['avis'][1]
        print(nb_bad, nb_good)

        to_remove = nb_good - nb_bad
        
        while(to_remove > 0):
            row: DataFrame = self.df.sample()
            index = row.first_valid_index()
            print(f"{to_remove}")

            if row['classe_bon_mauvais'][index] == 1:
                self.df.drop(index, inplace=True)
                to_remove -= 1
        
        d = self.df.groupby(['classe_bon_mauvais'], as_index=False).count()
        nb_bad = d['avis'][0]
        nb_good = d['avis'][1]
        print("2", nb_bad, nb_good)

    def save_data(self, path):
        self.df.to_csv(path, index=False)


> ## Pre-processing
> Données avant pre processing

In [13]:
import pandas as pd

df = pd.read_csv('dataset/csv/dataset_original.csv')

df['avis'][3]

"Aller, un bon 10 parce que ca reste jouable, mais systeme de paiement si tu veut plus, un jeu copier coller de l'ancienne version ,et on recommence ! pas besoin d'en dire plus"

> Données après pre-processing :

In [14]:
import pandas as pd

df = pd.read_csv('dataset/csv/dataset_0-1.csv')

df['avis'][3]

"aller bon parce reste jouable mais systeme paiement si veut plus copier coller l'ancienne version recommence pas besoin d'en dire plus"

> Nous avons calculer les 30 mots les plus utilisés que l'on a par la suite enlevé. parmis eux :

```
"c'est", "ces", "ses", "s'est", "a", "de", "du", "et", "le", "les", "un", "une", "pour", "sur", "etc", "est", "c",
'la', "jeu", "que", "des", "en", "ce", "qu", "ca", "y", "je", "sa", "son","au", "ai", "mon", "ma", "mes", "qui", "je", 
"tu", "il", "ils", "elles", "elle", "vous", "nous","qu'il", "qu'elle", "qu'ils", "qu'elles", "qu'on", "on", "se", "par"
```

> Les avis trop longs posent aussi problèmes. Certains avis comportaient plus de 1000 mots, pour une moyenne beaucoup plus basse. <br>
> Cela pose problème pour l'entrainement mais aussi pour la représentation des avis.

<br>

> ## Classes des notes
> > Pour représenter les notes, nous avons instaurer 2 classes différentes, 0 pour les notes < 12, et 1 pour les notes >= 12.

<br>

> Dataset 2 classes :

In [15]:
import pandas as pd

df = pd.read_csv('dataset/csv/dataset_0-1.csv')

df.sample(frac=1)

,classe_bon_mauvais,avis
23587,0,navet siècle
19275,0,selon moi aucun ne mérite donc cela lui donne
19606,1,ac iv black flag réussite trouve l'ambiance at...
8150,0,qu'est-ce si spécial durée vie mode solo plus ...
3308,1,graphismes vieillissent pas trop mal histoire ...
...,...,...
12249,0,suis généreux pc n'y pas différence mais j'ai ...
17979,1,très bien mais comprends l'intérêt foutre mont...
1745,0,nul car toujours dans couloir l'on perd tout t...
35057,0,bon donne moyenne parce ff mais honnêtement lo...


> Dataset 4 classes :

In [16]:
import pandas as pd

df = pd.read_csv('dataset/csv/dataset_0-3.csv')

df.sample(frac=1)

,classe_bon_mauvais,avis
8386,2,ah call of duty mais où donc passé cette série...
26616,0,ah faut arrêter avec maintenant serait peut-êt...
24699,0,un zelda insipide on prend fait depuis décenni...
5359,3,ce vraiment simulation combat top top concerne...
19623,3,assez bon avec beaux graphismes addiction plut...
...,...,...
26252,2,un bon jeux promet belle chose attendre pour l...
25813,3,ah skyrim un j'ai tant attendu
2003,3,un très bon halo le problème one manque puissa...
19294,3,j'ai découvert jv com n'ai vraiment jamais prê...


> ## Corretions des mots
> > Afin de tenir comptes des mots mal orthographiés, nous avons utilisés la librairie [spellchecker](https://pypi.org/project/pyspellchecker/). <br>
> > Malheureusement nous avons fait une erreur dans le code. En effet nous avons transformé tous les points et toutes les virgules par du vide. <br>
> > Nous avons perdu par ce biais beaucoup de mots car inutilisables pour la méthode Word2Vec.

<br>

> 